In [8]:
from docplex.mp.model import Model
import pandas as pd
import numpy as np
import warnings
from docplex.util.environment import get_environment
from datetime import datetime
warnings.filterwarnings("ignore")
from preprocessing import PreprocessData

pd.set_option('max_colwidth', 100)

In [9]:
class MakeData(object):
    def __init__(self, model, data, col):
        self.model = model
        self.data = data
        self.col = col
    def make_params(self, inds):
        try:
            new_data = self.data[self.col].groupby(by=inds).aggregate(lambda x: self.model.sum(x)).reset_index().rename(
                columns={self.col: self.col.replace(self.col[: self.col.index('_')], 'sum')})
        except ValueError:
            new_data = self.data[self.col].groupby(by=inds).aggregate(lambda x: self.model.sum(x)).reset_index()
        return new_data

    def make_df(self, param, ind='time_period'):
        if ind == 'time_period':
            new_df = pd.DataFrame(columns=[ind]+self.col)
        else:
            new_df = pd.DataFrame(columns=self.col+[ind])
        ele = [[i] * len(self.data) for i in param]
        add_col = [i for e in ele for i in e]
        for t in param:
            new_df = pd.concat([new_df, self.data])
        new_df[ind] = add_col
        new_df.reset_index(inplace=True, drop=True)
        return new_df

    def split_cols(self, trans_col):
        return self.data[self.col.strip('ID')+'_'+trans_col.strip('ID')].str.split(',', expand=True).rename(
            columns={0:self.col, 1:trans_col}).apply(pd.to_numeric, errors='coerce')

    def get_dist(self, dist_name):
        df = self.data.loc[(self.data['ID1_type']==self.col[0]) & (self.data['ID2_type']==self.col[1])][
            ['ID1', 'ID2', 'distances']]
        df.rename(columns={'ID1': self.col[0], 'ID2': self.col[1], 'distances': dist_name}, inplace=True)
        for i in self.col:
            if not df[i].isnull().any():
                df[i] = df[i].astype(int)
            else:
                continue
        return df

def get_params(orig_df, param_df, params, ind=list(), repetitive=False, dropped=False):
    #param_dict = {}
    param_ = param_df[ind+params].set_index(ind)
    orig_cols = orig_df.columns.tolist()
    param_cols = param_df.columns.tolist()
    incompatible_name = list(set(orig_cols).intersection(set(param_cols)))
    if len(incompatible_name) > 0:
        for n in incompatible_name:
            param_.rename(columns={n: n+'_'}, inplace=True)
    # 29/08/2022 -- added for repetitive keys when calculating precedences
    if repetitive:
        param_dict = dict()
        param_dict[params[0]] = param_.groupby(ind)[params[0]].apply(list).to_dict()
    else:
        param_dict = param_.to_dict()
    orig_df.set_index(ind, inplace=True)
    for p in param_dict.keys():
        orig_df[p] = orig_df.index.to_numpy()
        orig_df[p] = orig_df[p].map(param_dict[p])
    orig_df.reset_index(inplace=True)
    if repetitive:
        orig_df = orig_df.explode(params[0])
    if dropped:
        orig_df.dropna(inplace=True)
    else:
        orig_df.fillna(0, inplace=True)
    return orig_df

# dump location priority for dumping
def outputDiagonal(lst, direction='secondary'):
    # To print Primary Diagonal
    if direction == 'primary':
        return [lst[i][i] for i in range(len(lst))]
    else:
        return [lst[i][len(lst)-1-i] for i in range(len(lst))]

def groupDiagonal(dim: dict()):
    namesList = ['lift', 'priority', 'locationID']
    total_arr = {}
    priority_arr = {}
    df_priority = pd.DataFrame()
    for dump in dim.keys():
        for i in range(len(dim[dump])):
            add = np.sum([j**2 for j in dim[dump][:i]])
            lst = [k + add for k in list(range(1, dim[dump][i]**2+1))]
            arr = np.array(lst).reshape(-1, dim[dump][i])
            priority_arr[i+1] = {}
            for l1 in range(1, dim[dump][i]+1):
                priority_arr[i+1][l1] = outputDiagonal(arr[:l1,:l1])
            for l2 in range(-dim[dump][i]+1, 0):
                priority_arr[i+1][-(-dim[dump][i]-l2)+dim[dump][i]] = outputDiagonal(arr[l2:, l2:])
            total_arr[i+1] = arr

        priority = pd.DataFrame.from_dict(priority_arr, orient='index').stack().explode().reset_index()
        priority.columns = namesList
        priority.insert(loc=0, column='dumpID', value=[dump]*len(priority))

        df_priority = pd.concat([df_priority, priority])
        df_priority['locationID'] = df_priority['locationID'].astype(int)
    return df_priority

In [10]:
def load_data(model, data_url, bin_num, grade_margin):

    #bin_precedence = pd.ExcelFile(data_url + "generatedFile.xlsx")
    
    model.grade_margin = grade_margin
    #model.df_blocks = pd.read_csv(data_url + "1.block_model_as_mined.csv")                        # block model
    model.df_blocks = pd.read_csv(data_url + "df_blocks_test2.csv")                        # block model
    model.df_blocks = model.df_blocks.loc[(model.df_blocks.z < 352.5)]
    #model.df_blocks = model.df_blocks.loc[model.df_blocks.mineID==3]
    model.df_blocks['vol'] = model.df_blocks['quantity'] / model.df_blocks['density']
    
    df_dumps = pd.read_csv(data_url + "2.waste_dump_new3.csv")                                # waste dumps
    df_dumps['dump'+'_'+'location'] = df_dumps[['dumpID', 'locationID']].apply(lambda x: ','.join(x.values.astype(str)),
                                                                               axis=1)
    
    stockpile_cap = pd.read_csv(data_url + "5.stockpile_capacity.csv")                   # stockpile bin capacity
    
    #model.dumping_priority = pd.read_csv(data_url + "dumping_priority.csv")
    cell_cnt_key = 'cell_cnt'
    cell_matrix = pd.DataFrame(df_dumps.groupby(['dumpID', 'z']).locationID.nunique().apply(np.sqrt)).rename(
        columns={'locationID': 'cell_cnt'})
    cell_matrix = cell_matrix.reset_index(level=['z'], drop=True)
    cell_matrix[cell_cnt_key] = cell_matrix[cell_cnt_key].astype('int')
    dump_priority = groupDiagonal(cell_matrix.groupby(level=0).agg(list).to_dict()[cell_cnt_key])
    
    # dumping_priority
    priority = dump_priority.rename(columns={'locationID':'precedenceID'})
    post_dumping_priority = dump_priority.rename(columns={'priority':'post_priority'})
    post_dumping_priority['priority'] = post_dumping_priority['post_priority'] - 1
    dumping_priority = get_params(post_dumping_priority, priority, params=['precedenceID'],
                                  ind=['dumpID', 'priority', 'lift'], repetitive=True)
    dumping_priority = get_params(dumping_priority, df_dumps, params=['volume'], ind=['dumpID', 'locationID'])
    model.dumping_priority = dumping_priority.loc[dumping_priority['precedenceID'] != 0]
    blockPreprocess = PreprocessData(model.df_blocks)
    dumpPreprocess = PreprocessData(df_dumps)
    # create precedent relationships
    df_block_precedence = blockPreprocess.precedence('mineID', 'blockID')
    df_waste_precedence = dumpPreprocess.precedence('dumpID', 'locationID')
    # 04/10 -- revised: only some of the dumps can accommodate active materials
    active_dumps = dumpPreprocess.detect_active('dumpID', 'locationID', [2])               # active info of dumps
    model.df_dumps = get_params(df_dumps, active_dumps, params=['fill_active'], ind=['dumpID', 'locationID'])
    # create suitable bins
    #df_bins = suitable_bin(model.df_blocks, stockpile_cap, bin_num, grade_margin)
    df_bins = blockPreprocess.suitable_bin(stockpile_cap, bin_num, grade_margin)
    model.df_bins = get_params(df_bins.drop(columns=['possible_bin']), model.df_blocks,
                               params=['if_active', 'quantity'], ind=['mineID', 'blockID'])
    model.df_bins = model.df_bins.loc[((model.df_bins.binID<0) & (model.df_bins.if_active==0)) | (model.df_bins.binID>0)]
    #model.df_bins = pd.read_csv(data_url + "df_bins.csv")

    # get the capacity of each bin
    model.bin_capacity_grade = model.df_bins[['binID', 'bin_capacity', 'avg_grade']].drop_duplicates()

    # get the quantity of each block for block precedent conditions
    model.mb_pre_cols = ['mineID', 'precedenceID', 'blockID']
    model.df_block_precedence = get_params(df_block_precedence, model.df_blocks.rename(columns={'blockID': 'precedenceID'}),
                                           params=['quantity'], ind=model.mb_pre_cols[:-1])
    # get the capacity of each dump location for dump location precedent conditions
    model.dl_pre_cols = ['dumpID', 'precedenceID', 'locationID']
    model.df_waste_precedence = get_params(df_waste_precedence, df_dumps.rename(columns={'locationID': 'precedenceID'}),
                                           params=['volume'],
                                           ind=model.dl_pre_cols[:-1])

    df_parameter = pd.read_csv(data_url + "8.other_parameters.csv")                     # parameters
    model.other_params = dict(zip(df_parameter.parameters, df_parameter.value))
    model.mining_cap = pd.read_csv(data_url + "3.mining_capacity_new.csv")                  # mining capacities
    model.processing_cap = pd.read_csv(data_url + "4.processing_capacity_new.csv")          # processing capacities
    df_distance = pd.read_csv(data_url + "6.facility_distances.csv")                    # facility distances
    # distances
    # i. distance_m_b_p
    model.distance_mbp_ind = ['mineID', 'processorID']
    model.distance_m_b_p = MakeData(model, df_distance, model.distance_mbp_ind).get_dist('distance_m_b_p')
    # ii. distance_m_b_s
    model.distance_mbs_ind = ['mineID', 'binID']
    model.distance_m_b_s = MakeData(model, df_distance, model.distance_mbs_ind).get_dist('distance_m_b_s')
    # iii. distance_m_b_d_l
    model.distance_mbdl_ind = ['mineID', 'dumpID']
    model.distance_m_b_d_l = MakeData(model, df_distance, model.distance_mbdl_ind).get_dist('distance_m_b_d_l')
    # iv. distance_s_p
    model.distance_sp_ind = ['binID', 'processorID']
    model.distance_s_p = MakeData(model, df_distance, model.distance_sp_ind).get_dist('distance_s_p')
    # iv. distance_s_d_l
    model.distance_sdl_ind = ['binID', 'dumpID']
    model.distance_s_d_l = MakeData(model, df_distance, model.distance_sdl_ind).get_dist('distance_s_d_l')

    # make processing cost and recovery rate into a single column
    pr_cols = ['mineID', 'blockID', 'grade', 'mining_cost', 'processing_cost_p1', 'processing_cost_p2',
               'recovery_rate_p1', 'recovery_rate_p2']
    processor_recovery = pd.wide_to_long(model.df_blocks[pr_cols],
                                         i=pr_cols[:4],
                                         stubnames=['processing_cost_p', 'recovery_rate_p'],
                                         j='processorID').set_axis(['processing_cost', 'recovery_rate'],
                                                                   axis='columns').reset_index()
    model.processor_recovery = get_params(processor_recovery, model.df_bins, params=['binID', 'avg_grade', 'grade'],
                                          ind=pr_cols[:2])
    model.pro_rec_rate = (model.processor_recovery.processing_cost/model.processor_recovery.recovery_rate).unique()

    model.rou = pd.read_csv(data_url + "7.swell_factor.csv")
    # data
    model.mine_block = model.df_blocks[['mineID', 'blockID', 'if_active', 'grade', 'density', 'wting',
                                        'mining_cost', 'quantity']]
    model.dump_location = model.df_dumps[['dumpID', 'locationID', 'dump_location', 'fill_active']]

In [11]:
def setup_data(model, grade_zero=False):
    # parameters
    n_processors = int(model.other_params['n_processors'])
    periods = int(model.other_params['time period'])
    metal_price = model.other_params['unit metal price']                       # $/g
    #metal_price = 2000  
    refining_cost = model.other_params['unit refining cost']                   # $/g
    delta_haulage_cost = model.other_params['delta haulage cost']              # $/tonne-bench (used when sent to waste dumps)
    haulage_cost = model.other_params['unit haulage cost']                     # $/tonne-km
    train_haulage_cost = model.other_params['train haulage cost (processing)'] # $/tonne-km
    rehandling_cost = model.other_params['unit stockpile rehandling cost']     # $/tonne
    rou = model.rou
    # define the lower bound of grade that allows sending materials to processors as "hard" cut-off grade
    model.grade_lower_bound = (model.pro_rec_rate/(metal_price-refining_cost)).min()
    # col indexes
    model.mb_cols = ['mineID', 'blockID']
    model.dl_cols = ['dumpID', 'locationID']
    model.mbs_cols = model.mb_cols + ['binID']
    model.mbdl_cols = model.mb_cols + model.dl_cols
    # data
    mine_block = model.mine_block
    dump_location = model.dump_location

    # excavated
    model.excavated = MakeData(model, mine_block[model.mb_cols], model.mb_cols).make_df(range(1, periods+1))
    # filled
    model.filled = MakeData(model, dump_location[model.dl_cols], model.dl_cols).make_df(range(1, periods+1))
    # t_m_b_p
    t_m_b = MakeData(model, mine_block[model.mb_cols], model.mb_cols).make_df(range(1, periods+1))
    m_b_p = MakeData(model, mine_block.loc[mine_block['grade']>=grade_margin, 
                                           model.mb_cols+['grade', 'mining_cost', 'quantity']], model.mb_cols).make_df(
        range(1, n_processors+1), ind='processorID')
    t_m_b_p = MakeData(model, m_b_p, m_b_p.columns.tolist()).make_df(range(1, periods+1))
    t_m_b_p = get_params(t_m_b_p, model.distance_m_b_p, params=['distance_m_b_p'], ind=model.distance_mbp_ind)
    model.t_m_b_p = get_params(t_m_b_p, model.processor_recovery,
                               params=['processing_cost', 'recovery_rate'], ind=model.mb_cols+['processorID'])
    # selling_price_m_b_p =
    # (metal_price-refinining_cost)*grade_m_b*recovery_m_b_p-mining_cost_m_b-haulage_cost_m_b_p-processing_cost_m_b_p
    model.t_m_b_p['selling_price'] = np.where(model.t_m_b_p['processorID']==1,
                                              (metal_price - refining_cost) * model.t_m_b_p.grade * model.t_m_b_p.recovery_rate - model.t_m_b_p.mining_cost - haulage_cost * model.t_m_b_p.distance_m_b_p - model.t_m_b_p.processing_cost,
                                              (metal_price - refining_cost) * model.t_m_b_p.grade * model.t_m_b_p.recovery_rate - model.t_m_b_p.mining_cost - (haulage_cost * (model.t_m_b_p.distance_m_b_p-40) + train_haulage_cost * 40) - model.t_m_b_p.processing_cost
                                              )
    # t_m_b_s
    t_m_b_s = MakeData(model, model.df_bins[model.mbs_cols+['density', 'if_active', 'quantity']],
                       model.mbs_cols).make_df(range(1, periods+1))
    t_m_b_s = get_params(t_m_b_s, model.distance_m_b_s, params=['distance_m_b_s'], ind=model.distance_mbs_ind)
    # 14/09/2022 -- revised: replace with the real distance to bin[-999] if grade=0
    '''t_m_b_s.distance_m_b_s = np.where(t_m_b_s.distance_m_b_s==0,
                                      model.distance_m_b_s.loc[model.distance_m_b_s.binID==-999].distance_m_b_s.tolist()[0],
                                      t_m_b_s.distance_m_b_s)'''
    model.t_m_b_s = get_params(t_m_b_s, model.processor_recovery, params=['mining_cost', 'grade', 'avg_grade'],
                               ind=model.mb_cols+['binID'])
    # selling_price_m_b_s =
    # -mining_cost_m_b-haulage_cost_m_b_s
    # 05/10/2022 -- revised: if the grade is less than the avg_grade, reduce selling price; otherwise, increase selling price.
    model.t_m_b_s['grade_diff'] = model.t_m_b_s['avg_grade'] - model.t_m_b_s['grade']
    #model.t_m_b_s['selling_price'] = -model.t_m_b_s.mining_cost - haulage_cost * model.t_m_b_s.distance_m_b_s - (metal_price - refining_cost) * model.t_m_b_s.grade_diff * 0.008
    model.t_m_b_s['selling_price'] = -model.t_m_b_s.mining_cost - haulage_cost * model.t_m_b_s.distance_m_b_s
    # t_m_b_d_l
    m_b_d_l_active = MakeData(model, mine_block.loc[(mine_block['if_active']==1) & 
                                                    (mine_block['grade']<round(model.grade_lower_bound+0.005,2)),
                                                    model.mb_cols+['density', 'grade', 'wting', 'if_active', 'quantity']], 
                              model.mb_cols).make_df(
        dump_location.loc[dump_location['fill_active']==1].dump_location.unique().tolist(), ind='dump_location')
    m_b_d_l_nonactive = MakeData(model, mine_block.loc[(mine_block['if_active']==0) & 
                                                       (mine_block['grade']<round(model.grade_lower_bound+0.005,2)),
                                                       model.mb_cols+['density', 'grade', 'wting', 'if_active', 'quantity']],
                                 model.mb_cols).make_df(dump_location.dump_location.unique().tolist(), ind='dump_location')
    m_b_d_l = pd.concat([m_b_d_l_active, m_b_d_l_nonactive])
    m_b_d_l = get_params(m_b_d_l, dump_location, params=['fill_active'], ind=['dump_location'])
    m_b_d_l = pd.concat([m_b_d_l.drop(columns='dump_location'),
                         MakeData(model, m_b_d_l, 'dumpID').split_cols('locationID')], axis=1)
    t_m_b_d_l = MakeData(model, m_b_d_l, m_b_d_l.columns.tolist()).make_df(range(1, periods+1))
    t_m_b_d_l = get_params(t_m_b_d_l, model.distance_m_b_d_l, params=['distance_m_b_d_l'], ind=model.distance_mbdl_ind)
    t_m_b_d_l = get_params(t_m_b_d_l, model.rou, params=['swell_factor'], ind=['wting'])
    model.t_m_b_d_l = get_params(t_m_b_d_l, model.processor_recovery, params=['mining_cost'], ind=model.mb_cols)
    # selling_price_m_b_d_l =
    # -mining_cost_m_b-haulage_cost_m_b_d_l
    # 04/10 -- revised incur additional fees when non-active to active locations
#     model.t_m_b_d_l['selling_price'] = np.where((model.t_m_b_d_l['if_active']==0) & (model.t_m_b_d_l['fill_active']==1),
#                                                 -model.t_m_b_d_l.mining_cost - haulage_cost * model.t_m_b_d_l.distance_m_b_d_l - 0.01,
#                                                 np.where((model.t_m_b_d_l['if_active']==0) & (model.t_m_b_d_l['dumpID']==1),
#                                                          -model.t_m_b_d_l.mining_cost - haulage_cost * model.t_m_b_d_l.distance_m_b_d_l - 0.3,
#                                                          -model.t_m_b_d_l.mining_cost - haulage_cost * model.t_m_b_d_l.distance_m_b_d_l
#                                                         )
#                                                )
    model.t_m_b_d_l['selling_price'] = -model.t_m_b_d_l.mining_cost - haulage_cost * model.t_m_b_d_l.distance_m_b_d_l
    
    # t_s_p
    bin_p = pd.DataFrame(model.df_bins.loc[model.df_bins.binID>0].binID.unique().tolist(), columns=['binID'])
    s_p = MakeData(model, bin_p, bin_p.columns.tolist()).make_df(range(1, n_processors+1), ind='processorID')
    t_s_p = MakeData(model, s_p, s_p.columns.tolist()).make_df(range(1, periods+1))
    t_s_p = get_params(t_s_p, model.distance_s_p, params=['distance_s_p'], ind=['processorID'])
    # selling_price_s_p =
    # (metal_price-refinining_cost)*grade_s*recovery_s_p-rehandling_cost-haulage_cost_s_p-processing_cost_s_p
    model.t_s_p = get_params(t_s_p, model.processor_recovery, params=['avg_grade', 'processing_cost', 'recovery_rate'],
                             ind=['binID', 'processorID'])
    model.t_s_p['selling_price'] = np.where(model.t_s_p['processorID']==1,
                                            (metal_price - refining_cost) * model.t_s_p.avg_grade * model.t_s_p.recovery_rate - rehandling_cost - haulage_cost * model.t_s_p.distance_s_p - model.t_s_p.processing_cost,
                                            (metal_price - refining_cost) * model.t_s_p.avg_grade * model.t_s_p.recovery_rate - rehandling_cost - (haulage_cost * (model.t_s_p.distance_s_p-40) + train_haulage_cost * 40) - model.t_s_p.processing_cost
                                            )
    # t_s_d_l
    if grade_zero:
        bin_d = pd.DataFrame(np.array([[0, 1, 1.7]]), columns=['binID', 'wting', 'density'])
    else:
        bin_d = model.df_bins.loc[model.df_bins['binID'].isin([-999,-998,-997]),
                                  ['binID', 'wting', 'density']].drop_duplicates()
    
    s_dl = MakeData(model, bin_d, bin_d.columns.tolist()).make_df(dump_location.loc[dump_location.dumpID==2].dump_location.unique().tolist(),
                                                                 ind='dump_location')
    s_d_l = get_params(s_dl, model.dump_location, params=['fill_active'], ind=['dump_location'])
    s_d_l = pd.concat([s_dl.drop(columns='dump_location'), MakeData(model, s_dl, 'dumpID').split_cols('locationID')], axis=1)

#     s_d_l = pd.concat([s_dl.drop(columns='dump_location'),
#                        MakeData(model, s_dl, 'dumpID').split_cols('locationID')], axis=1)
    t_s_d_l = MakeData(model, s_d_l, s_d_l.columns.tolist()).make_df(range(1, periods+1))
    t_s_d_l = get_params(t_s_d_l, rou, params=['swell_factor'], ind=['wting'])

    '''
    # 31/10 -- revised add new columns 'swelled_density' to eliminate duplication and delete original columns 'swell_factor'
    # & 'density'
    t_s_d_l['swelled_density'] = t_s_d_l['density'] / t_s_d_l['swell_factor']
    t_s_d_l.drop(['density', 'swell_factor'], axis=1, inplace=True)
    t_s_d_l.drop_duplicates(inplace=True)
    '''
    model.t_s_d_l = get_params(t_s_d_l, model.distance_s_d_l, params=['distance_s_d_l'], ind=model.distance_sdl_ind)
    # selling_price_s_d_l =
    # -rehandling_cost-haulage_cost_s_d_l
    model.t_s_d_l['selling_price'] = -rehandling_cost - haulage_cost * model.t_s_d_l.distance_s_d_l

    # add name for each dataframe
    model.excavated['name'] = model.excavated.apply(lambda x: "excavated_time%d_mine%d_block%d" %
                                                              (x['time_period'], x['mineID'], x['blockID']), axis=1)
    model.filled['name'] = model.filled.apply(lambda x: "filled_time%d_dump%d_loc%d" %
                                              (x['time_period'], x['dumpID'], x['locationID']), axis=1)
    model.t_m_b_p['name'] = model.t_m_b_p.apply(lambda x: "time%d_mine%d_block%d_processor%d" %
                                                (x['time_period'], x['mineID'], x['blockID'], x['processorID']), axis=1)
    model.t_m_b_s['name'] = model.t_m_b_s.apply(lambda x: "time%d_mine%d_block%d_bin%d" %
                                                (x.time_period, x.mineID, x.blockID, x.binID), axis=1)
    model.t_m_b_d_l['name'] = model.t_m_b_d_l.apply(lambda x: "time%d_mine%d_block%d_dump%d_loc%d" %
                                                    (x.time_period, x.mineID, x.blockID, x.dumpID, x.locationID), axis=1)
    model.t_s_p['name'] = model.t_s_p.apply(lambda x: "time%d_bin%d_processor%d" %
                                            (x.time_period, x.binID, x.processorID), axis=1)
    model.t_s_d_l['name'] = model.t_s_d_l.apply(lambda x: "time%d_bin%d_dump%d_den%.2f_loc%d" %
                                                (x.time_period, x.binID, x.dumpID, x.density, x.locationID), axis=1)

In [12]:
def setup_variables(model):
    discount = model.other_params['discount_rate']
    # auxiliary function to create variables from a row
    def create_var(model, df, var_name, var_type='cont'):
        df.set_index('name', inplace=True)
        if var_type=='cont':
            df[var_name] = df.apply(lambda r: model.continuous_var(name="tons_%s" % r.name, lb=0), axis=1)
            # make_cont_var(r, "tons_%s"), axis=1)
        else:
            df[var_name] = df.apply(lambda r: model.binary_var(name="%s" % r.name), axis=1)
            # make_cat_var(r, "%s"), axis=1)
        df.reset_index(drop=True, inplace=True)
        return df

    # excavated
    model.excavated = create_var(model, model.excavated, "excavated", var_type='cat')
    model.excavated['sum_excavated'] = model.excavated.groupby(['mineID', 'blockID']).excavated.apply(lambda x: x.cumsum())
    # filled
    model.filled = create_var(model, model.filled, "filled", var_type='cat')
    # t_m_b_p
    model.t_m_b_p = create_var(model, model.t_m_b_p, "tons_t_m_b_p")
    model.t_m_b_p['profit'] = model.t_m_b_p.tons_t_m_b_p * model.t_m_b_p.selling_price / ((1 + discount/100)**model.t_m_b_p.time_period)
    # t_m_b_s
    model.t_m_b_s = create_var(model, model.t_m_b_s, "tons_t_m_b_s")
    model.t_m_b_s['sum_m_b_s'] = model.t_m_b_s.groupby(['mineID', 'blockID']).tons_t_m_b_s.apply(lambda x: x.cumsum())
    model.t_m_b_s['profit'] = model.t_m_b_s.tons_t_m_b_s * model.t_m_b_s.selling_price / ((1 + discount/100)**model.t_m_b_s.time_period)
    # t_m_b_d_l
    model.t_m_b_d_l = create_var(model, model.t_m_b_d_l, "tons_t_m_b_d_l")
    model.t_m_b_d_l['vol_t_m_b_d_l'] = (model.t_m_b_d_l['swell_factor']*model.t_m_b_d_l['tons_t_m_b_d_l'])/model.t_m_b_d_l['density']
    model.t_m_b_d_l['sum_m_b_filled'] = model.t_m_b_d_l.groupby(['mineID', 'blockID', 'dumpID']).vol_t_m_b_d_l.apply(
        lambda x: x.cumsum())
    model.t_m_b_d_l['profit'] = model.t_m_b_d_l.tons_t_m_b_d_l * model.t_m_b_d_l.selling_price / ((1 + discount/100)**model.t_m_b_d_l.time_period)
    # t_s_p
    model.t_s_p = create_var(model, model.t_s_p, "tons_t_s_p")
    model.t_s_p['sum_s_p'] = model.t_s_p.groupby(['binID', 'processorID']).tons_t_s_p.apply(lambda x: x.cumsum())
    model.t_s_p['profit'] = model.t_s_p.tons_t_s_p * model.t_s_p.selling_price / ((1 + discount/100)**model.t_s_p.time_period)
    # t_s_d_l
    model.t_s_d_l = create_var(model, model.t_s_d_l, "tons_t_s_d_l")
    model.t_s_d_l['vol_t_s_d_l'] = (model.t_s_d_l['swell_factor'] * model.t_s_d_l['tons_t_s_d_l']) / model.t_s_d_l['density']
    model.t_s_d_l['profit'] = model.t_s_d_l.tons_t_s_d_l * model.t_s_d_l.selling_price / ((1 + discount/100)**model.t_s_d_l.time_period)
    model.t_s_d_l = model.t_s_d_l.groupby(['binID', 'dumpID', 'locationID', 'time_period', 'fill_active', 'distance_s_d_l', 
                                           'selling_price'])['tons_t_s_d_l', 'vol_t_s_d_l', 'profit'].sum().reset_index()
    model.t_s_d_l['sum_s_filled'] = model.t_s_d_l.groupby(['binID', 'dumpID', 'locationID']).vol_t_s_d_l.apply(lambda x: 
                                                                                                               x.cumsum())
    model.t_s_d_l['sum_s_d_l'] = model.t_s_d_l.groupby(['binID', 'dumpID', 'locationID']).tons_t_s_d_l.apply(lambda x: 
                                                                                                             x.cumsum())

In [13]:
# constraints -> mdl.add_constraint
def setup_constraints(model):
    periods = int(model.other_params['time period'])
    grade_lower_bound = model.grade_lower_bound
    mine_block = model.mine_block
    dump_location = model.dump_location
    #dumping_priority = model.dumping_priority

    excavated = model.excavated
    filled = model.filled
    t_m_b_p = model.t_m_b_p
    t_m_b_s = model.t_m_b_s
    t_m_b_d_l = model.t_m_b_d_l
    t_s_p = model.t_s_p
    t_s_d_l = model.t_s_d_l

    def pin_period(df, col, v):
        cond = (df[col] <= v)
        return cond

    def precedences(model, df, col, const=['time_period'], ls=list()):
        IDs = []
        ts = []
        values = []
        col_name = ls + const + [col]
        ls_len = len(ls)
        for item, item_flow in df.groupby(level=ls):
            for t in range(1, periods+1):
                value = model.sum(item_flow[pin_period(item_flow, 'time_period', t)][col])
                if type(item) != tuple:
                    ID = item
                else:
                    ID = list(item)
                IDs.append(ID)
                ts.append(t)
                values.append(value)

        trans = np.ravel(IDs, order='F').reshape(ls_len,-1).T
        values = np.array(values).reshape(-1,1)
        ts = np.array(ts).reshape(-1,1)
        merged = np.concatenate((trans, ts, values), axis=1)
        df_final = pd.DataFrame(merged, columns=col_name)
        df_final.rename(columns={col_name[-1]:col_name[-1].replace(col_name[-1][: col_name[-1].index('_')], 'sum')},
                        inplace=True)
        return df_final

    # rule: active waste rocks don't go to the waste stockpile bin
    # 30/08/2022 -- revised: no constraint if there is no block with grade 0
    number_of_active_constraints = 0
    for r in t_m_b_s.itertuples():
        if len(model.df_bins.loc[model.df_bins.avg_grade==0].index.tolist()) == 0:
            pass
        elif r.if_active == 1 and r.binID == 0:
            model.add_constraint(r.tons_t_m_b_s == 0)
            number_of_active_constraints += 1
    print("#waste stockpile non-active constraints: {}".format(number_of_active_constraints))
    # model.print_information()
    # 1.1 reserve constraint
    number_of_reserve_constraints = 0
    reserve_level_cols = ['mineID', 'blockID', 'time_period']
    r1 = MakeData(model, t_m_b_p.set_index(reserve_level_cols), 'tons_t_m_b_p').make_params(reserve_level_cols)
    r2 = MakeData(model, t_m_b_s.set_index(reserve_level_cols), 'tons_t_m_b_s').make_params(reserve_level_cols)
    r3 = MakeData(model, t_m_b_d_l.set_index(reserve_level_cols), 'tons_t_m_b_d_l').make_params(reserve_level_cols)
    # reserve = get_params(reserve, excavated, params=['excavated'], ind=)
    reserve = get_params(excavated[['time_period', 'mineID', 'blockID', 'excavated']], 
                         mine_block[['mineID', 'blockID', 'quantity']], 
                         params=['quantity'], ind=['mineID', 'blockID'], dropped=False)
    reserve = get_params(reserve, r1, params=['sum_t_m_b_p'], ind=reserve_level_cols, dropped=False)
    reserve = get_params(reserve, r2, params=['sum_t_m_b_s'], ind=reserve_level_cols, dropped=False)
    reserve = get_params(reserve, r3, params=['sum_t_m_b_d_l'], ind=reserve_level_cols, dropped=False)

    for r in reserve.itertuples():
        model.add_constraint(r.sum_t_m_b_p + r.sum_t_m_b_s + r.sum_t_m_b_d_l == r.excavated * r.quantity)
        number_of_reserve_constraints += 1
    print("#reserve constraints: {}".format(number_of_reserve_constraints))
    # model.print_information()
    # 1.2 only-one-period mining
    for e in excavated.loc[excavated.time_period==periods].itertuples():
        model.add_constraint(e.sum_excavated <= 1)
    # 2. mining capacity
    number_of_miningCapacity_constraints = 0
    #mining_level_cols = ['mineID', 'time_period']
#     mining_level_cols = ['time_period']
#     mc1 = MakeData(model, t_m_b_p.set_index(mining_level_cols), 'tons_t_m_b_p').make_params(mining_level_cols)
#     mc2 = MakeData(model, t_m_b_s.set_index(mining_level_cols), 'tons_t_m_b_s').make_params(mining_level_cols)
#     mc3 = MakeData(model, t_m_b_d_l.set_index(mining_level_cols), 'tons_t_m_b_d_l').make_params(mining_level_cols)

#     mineCap = mc1.join(mc2.sum_t_m_b_s).join(mc3.sum_t_m_b_d_l)
    mineCap = excavated[['time_period', 'mineID', 'blockID', 'excavated']]
    mineCap = get_params(mineCap, mine_block, params=['quantity'], ind=['mineID', 'blockID'])
    mineCap['excavated_quantity'] = mineCap['excavated'] * mineCap['quantity']
    mine_params = ['lower_bound', 'upper_bound']
    #mine_ind = ['mineID']
    #mineCap = get_params(mineCap, model.mining_cap, params=mine_params, ind=mine_ind)
    mineCap['lower_bound'] = 0
    mineCap['upper_bound'] = 950000
    mineCap = MakeData(model, mineCap.set_index(['time_period']+mine_params), 'excavated_quantity').make_params(
        ['time_period']+mine_params)

    for m in mineCap.itertuples():
        if m.time_period < periods:
            model.add_constraint(m.sum_quantity <= m.upper_bound)
            model.add_constraint(m.sum_quantity >= m.lower_bound)
            number_of_miningCapacity_constraints += 2
        else:
            model.add_constraint(m.sum_quantity <= m.upper_bound)
            number_of_miningCapacity_constraints += 1
    print("#mining capacity constraints: {}".format(number_of_miningCapacity_constraints))
    # model.print_information()
    # 3. processing capacity
    number_of_processorCapacity_constraints = 0
    bin_ind = ['binID']
    processing_level_cols = ['processorID', 'time_period']
    pc1 = MakeData(model, t_m_b_p.set_index(processing_level_cols), 'tons_t_m_b_p').make_params(processing_level_cols)
    pc2 = MakeData(model, t_s_p.set_index(processing_level_cols), 'tons_t_s_p').make_params(processing_level_cols)
    processorCap = pc1.join(pc2.sum_t_s_p)
    # processorCap = reserve[['']]

    processor_params = ['lower_bound', 'upper_bound']
    processor_ind = ['processorID']
    processorCap = get_params(processorCap, model.processing_cap, params=processor_params, ind=processor_ind)

    for p in processorCap.itertuples():
        if p.time_period < periods:
            model.add_constraint(p.sum_t_m_b_p + p.sum_t_s_p <= p.upper_bound)
            model.add_constraint(p.sum_t_m_b_p + p.sum_t_s_p >= p.lower_bound)
            number_of_processorCapacity_constraints += 2
        else:
            model.add_constraint(p.sum_t_m_b_p + p.sum_t_s_p <= p.upper_bound)
            number_of_processorCapacity_constraints += 1
    print("#processing capacity constraints: {}".format(number_of_processorCapacity_constraints))
    # model.print_information()
    # 4. dumping capacity
    number_of_dumpLocCapacity_constraints = 0
    dumping_level_cols = ['dumpID', 'locationID']
    dlc1 = MakeData(model, t_m_b_d_l.set_index(dumping_level_cols), 'vol_t_m_b_d_l').make_params(dumping_level_cols)
    dlc2 = MakeData(model, t_s_d_l.set_index(dumping_level_cols), 'vol_t_s_d_l').make_params(dumping_level_cols)
    dumpLocCap = dlc1.join(dlc2.sum_t_s_d_l)

    dumpLoc_params = ['volume']
    dumpLoc_ind = ['dumpID', 'locationID']
    #dump_vol = pd.DataFrame(model.df_dumps.groupby('dumpID').volume.sum().reset_index())
    dumpLocCap = get_params(dumpLocCap, model.df_dumps, params=dumpLoc_params, ind=dumpLoc_ind)
    ## =========================================================================================================
    ## except the waste bin, other bins cannot go to dump????? -- to be decided by more reasonable cutoff grade
    ## =========================================================================================================
    for dl in dumpLocCap.itertuples():
        if dl.dumpID == 2:
            #model.add_constraint(dl.sum_t_m_b_d_l + dl.sum_t_s_d_l >= dl.volume - 10000) 
            model.add_constraint(dl.sum_t_m_b_d_l + dl.sum_t_s_d_l <= dl.volume) 
        else:
            model.add_constraint(dl.sum_t_m_b_d_l + dl.sum_t_s_d_l <= dl.volume)
        # model.add_constraint(dl.sum_t_m_b_d_l + dl.sum_t_s_d_l <= dl.volume)
        number_of_dumpLocCapacity_constraints += 1
    print("#dumping capacity constraints: {}".format(number_of_dumpLocCapacity_constraints))
    # model.print_information()
    # 5. mining precedence
    number_of_blockPrecedent_constraints = 0
    block_level_cols = ['mineID', 'blockID']
    blockPred_level_cols = ['mineID', 'precedenceID']

    blockPre = pd.merge(model.df_block_precedence, excavated, 
                        left_on=blockPred_level_cols, right_on=block_level_cols)
    blockPre = blockPre.drop(columns=['blockID_y', 'excavated'])
    blockPre.rename(columns={'blockID_x': 'blockID', 'sum_excavated':'pre_sum_excavated'}, inplace=True)
    blockPre = pd.merge(blockPre, excavated, on=block_level_cols+['time_period'])

    for block_flow in blockPre.itertuples():
        model.add_constraint(block_flow.excavated <= block_flow.pre_sum_excavated)
        number_of_blockPrecedent_constraints += 1
    print("#block precedence constraints: {}".format(number_of_blockPrecedent_constraints))
    # model.print_information()
    # 6. dump location precedence
    # 6.1 fill the current dump location if its precedent dump locations are all fully filled
    number_of_dumpLocPrecedent_constraints = 0
    dumpLocPred_level_cols = ['dumpID', 'locationID']
    dumpLocPre_t_m_b_d_l = t_m_b_d_l.rename(columns={'locationID':'precedenceID'})
    dumpLocPre_t_s_d_l = t_s_d_l.rename(columns={'locationID':'precedenceID'})
    dumpLocPre_t_m_b_d_l = get_params(dumpLocPre_t_m_b_d_l, model.df_waste_precedence,
                                      params=model.dl_pre_cols[-1:], ind=model.dl_pre_cols[:-1], repetitive=True, dropped=True)
    dumpLocPre_t_s_d_l = get_params(dumpLocPre_t_s_d_l, model.df_waste_precedence,
                                    params=model.dl_pre_cols[-1:], ind=model.dl_pre_cols[:-1], repetitive=True, dropped=True)
    dumpLocPre_m_b = precedences(model, dumpLocPre_t_m_b_d_l.set_index(model.dl_pre_cols),
                                 col='vol_t_m_b_d_l', ls=dumpLocPred_level_cols)
    dumpLocPre_s = precedences(model, dumpLocPre_t_s_d_l.set_index(model.dl_pre_cols),
                               col='vol_t_s_d_l', ls=dumpLocPred_level_cols)
    dumpLocPre = dumpLocPre_m_b.join(dumpLocPre_s.sum_t_s_d_l)
    dumpLocPre.dumpID = dumpLocPre.dumpID.astype(int)
    dumpLocPre.locationID = dumpLocPre.locationID.astype(int)
    # get precedent volume
    dumpLocPre_volume = model.df_waste_precedence.groupby(by=dumpLocPred_level_cols)['volume'].sum().reset_index()
    dumpLocPre_volume = pd.merge(dumpLocPre_volume, filled, on=dumpLocPred_level_cols)

    for dumpLoc_flow in dumpLocPre.join(dumpLocPre_volume[['volume', 'filled']]).itertuples():
        model.add_constraint(dumpLoc_flow.sum_t_m_b_d_l + dumpLoc_flow.sum_t_s_d_l >=
                             dumpLoc_flow.volume * dumpLoc_flow.filled)
        number_of_dumpLocPrecedent_constraints += 1
    print("#dump location precedence constraints: {}".format(number_of_dumpLocPrecedent_constraints))
    # model.print_information()
    # 6.2 filling capacity of the current dump location should not exceed its volume == dumping capacity satisfied
    #     in each time period
    number_of_dumpLocNow_constraints = 0
    dumpLocNow_m_b = precedences(model, t_m_b_d_l.set_index(dumpLocPred_level_cols+['mineID', 'blockID']),
                                 col='vol_t_m_b_d_l', ls=dumpLocPred_level_cols)
    dumpLocNow_s = precedences(model, t_s_d_l.set_index(dumpLocPred_level_cols+bin_ind), col='vol_t_s_d_l',
                               ls=dumpLocPred_level_cols)
    ####### running slow -- needs to be modified!!! => pd.concat()
    dumpLocNow = dumpLocNow_m_b.join(dumpLocNow_s.sum_t_s_d_l)
    dumpLocNow = get_params(dumpLocNow, model.df_dumps, params=['volume'], ind=dumpLocPred_level_cols)
    dumpLocNow = get_params(dumpLocNow, filled, params=['filled'], ind=dumpLocPred_level_cols+['time_period'])

    for dumpLoc_flow in dumpLocNow.itertuples():
        model.add_constraint(dumpLoc_flow.sum_t_m_b_d_l + dumpLoc_flow.sum_t_s_d_l <=
                             dumpLoc_flow.volume * dumpLoc_flow.filled)
        number_of_dumpLocNow_constraints += 1
    print("#dump location now constraints: {}".format(number_of_dumpLocNow_constraints))
    model.print_information()
#     7. dump location priority
#     fill the current dump location if its previous dump locations are all fully filled
#     number_of_dumpLocPriority_constraints = 0
#     dumpLocPred_level_cols = ['dumpID', 'locationID']
#     dumpLocPre_t_m_b_d_l = t_m_b_d_l.rename(columns={'locationID':'precedenceID'})
#     dumpLocPre_t_m_b_d_l = get_params(dumpLocPre_t_m_b_d_l, dumping_priority,
#                                       params=model.dl_pre_cols[-1:], ind=model.dl_pre_cols[:-1], repetitive=True, dropped=True)
#     dumpLocPre = precedences(model, dumpLocPre_t_m_b_d_l.set_index(model.dl_pre_cols),
#                              col='density_t_m_b_d_l', ls=dumpLocPred_level_cols)
#     dumpLocPre.dumpID = dumpLocPre.dumpID.astype(int)
#     dumpLocPre.locationID = dumpLocPre.locationID.astype(int)
#     # get precedent volume
#     dumpLocPre_volume = dumping_priority.groupby(by=dumpLocPred_level_cols)['volume'].sum().reset_index()
#     dumpLocPre_volume = pd.merge(dumpLocPre_volume, filled, on=dumpLocPred_level_cols)

#     for dumpLoc_flow in dumpLocPre.join(dumpLocPre_volume[['volume', 'filled']]).itertuples():
#         model.add_constraint(dumpLoc_flow.sum_t_m_b_d_l >= dumpLoc_flow.volume * dumpLoc_flow.filled)
#         number_of_dumpLocPriority_constraints += 1
#     print("#dump location priority constraints: {}".format(number_of_dumpLocPriority_constraints))
    # 8. stockpile capacity
    number_of_binCapacity_constraints = 0
    bin_level_cols = ['binID']
    bin_params = ['bin_capacity']
    bin_m_b = precedences(model, t_m_b_s.set_index(model.mb_cols+bin_ind), col='tons_t_m_b_s', ls=bin_level_cols)
    bin_s_p = precedences(model, t_s_p.set_index(bin_ind+processor_ind), col='tons_t_s_p', ls=bin_level_cols)
    bin_s_d = precedences(model, t_s_d_l.set_index(bin_ind+dumpLoc_ind), col='tons_t_s_d_l', ls=bin_level_cols)
    binCap = bin_m_b.merge(bin_s_p, how='left').merge(bin_s_d, how='left')
    binCap = get_params(binCap, model.bin_capacity_grade, params=bin_params, ind=bin_ind)     

    for bin_flow in binCap.itertuples():
        '''
        if bin_flow.time_period == 1:
            model.add_constraint(bin_flow.sum_t_s_p + bin_flow.sum_t_s_d_l == 0)
        '''
        model.add_constraint(bin_flow.sum_t_m_b_s - bin_flow.sum_t_s_p - bin_flow.sum_t_s_d_l >= 0)
        model.add_constraint(bin_flow.sum_t_m_b_s - bin_flow.sum_t_s_p - bin_flow.sum_t_s_d_l <= bin_flow.bin_capacity)
        number_of_binCapacity_constraints += 2
    print("#Stockpile bin capacity constraints: {}".format(number_of_binCapacity_constraints))

    # 16/09/2022 -- revised: no material flow from stockpiles when time period = 1
    number_of_voidBin_constraints = 0
    bin_flow_year0 = binCap.loc[binCap.time_period==1]
    for bin_flow in bin_flow_year0.itertuples():
        model.add_constraint(bin_flow.sum_t_s_p + bin_flow.sum_t_s_d_l == 0)
        number_of_voidBin_constraints += 1
    print("#Stockpile bin void flow constraints: {}".format(number_of_voidBin_constraints))
    # model.print_information()
    # 9. grade blending
    number_of_gradeBlend_constraints = 0
    gradeBlend = pd.concat([t_m_b_p[processing_level_cols+['grade', 'tons_t_m_b_p']].rename(columns={'tons_t_m_b_p': 'tons'}),
                            t_s_p[processing_level_cols+['avg_grade', 'tons_t_s_p']].rename(columns={'tons_t_s_p': 'tons',
                                                                                                     'avg_grade': 'grade'})],
                           ignore_index=True)
    gradeBlend['grade_tons'] = gradeBlend['grade'] * gradeBlend['tons']

    for p, grade in gradeBlend.groupby(by=processing_level_cols):
        model.add_constraint(model.sum(grade.grade_tons) >= grade_lower_bound * model.sum(grade.tons))
        number_of_gradeBlend_constraints += 1
    print("#Grade blending constraints: {}".format(number_of_gradeBlend_constraints))

    model.print_information()


def setup_objective(model, data_url):
    total_t_m_b_p_revenues = model.sum(model.t_m_b_p['profit'])
    total_t_m_b_s_revenues = model.sum(model.t_m_b_s['profit'])
    total_t_m_b_d_l_revenues = model.sum(model.t_m_b_d_l['profit'])
    total_t_s_p_revenues = model.sum(model.t_s_p['profit'])
    total_t_s_d_l_revenues = model.sum(model.t_s_d_l['profit'])
    model.add_kpi(total_t_m_b_p_revenues, "Total t_m_b_p revenues")
    model.add_kpi(total_t_m_b_s_revenues, "Total t_m_b_s revenues")
    model.add_kpi(total_t_m_b_d_l_revenues, "Total t_m_b_d_l revenues")
    model.add_kpi(total_t_s_p_revenues, "Total t_s_p revenues")
    model.add_kpi(total_t_s_d_l_revenues, "Total t_s_d_l revenues")

    model.maximize(total_t_m_b_p_revenues + total_t_m_b_s_revenues + total_t_m_b_d_l_revenues +
                   total_t_s_p_revenues + total_t_s_d_l_revenues)
    # save the lp model
    #model.export_as_lp(data_url+"final_model_ji.lp")
    #model.export_as_lp("C:/Users/20937337/Desktop/final_model.lp")


def print_information(model):
    model.print_information()
    model.report_kpis()

def solve(model, **kwargs):
    # Here, we set the number of threads for CPLEX to 2 and set the time limit to 2mins.
    #model.parameters.threads = 2
    #model.parameters.timelimit = 120
    model.parameters.mip.tolerances.mipgap.set(float(0.1))
    model.parameters.mip.strategy.file.set(3)
    sol = model.solve(log_output=True, SearchType='DepthFirst',**kwargs)
    if sol is not None:
        print("solution for a cost of {}".format(model.objective_value))
        #print_information(model)
        #print_solution(model)
        return sol, model.objective_value
    else:
        print("* model is infeasible")
        return None

def get_solution(data, sol, var, trans=False):
    data['var_value'] = sol.get_values(data[var])
    if trans:
        data['var_value'] = data['var_value'].astype(int)
    data['var_value'] = data['var_value'].apply(lambda x: 0 if x < 1e-5 else x)
    return data


def build(bin_num, data_url, grade_margin, context=None, verbose=False, **kwargs):
    #-----start------
    mdl = Model("mine_prod", context=context, **kwargs)
    loading_t0 = datetime.now()
    load_data(mdl, data_url, bin_num, grade_margin)
    #print('loading data completed!')
    #return mdl, loading_t0

    setup_data(mdl)

    preprocessing_t1 = datetime.now()
    print("data loading time: ", (preprocessing_t1 - loading_t0).seconds/60)
    setup_variables(mdl)
    variables_setting_t2 = datetime.now()
    print("data preprocessing time: ", (variables_setting_t2 - preprocessing_t1).seconds/60)
    setup_constraints(mdl)
    constraints_setting_t3 = datetime.now()
    print("variables setting time: ", (constraints_setting_t3 - variables_setting_t2).seconds/60)
    setup_objective(mdl, data_url)
    objective_setting_t4 = datetime.now()
    print("objective setting time: ", (objective_setting_t4 - constraints_setting_t3).seconds/60)
    print("model completion time: ", (objective_setting_t4 - loading_t0).seconds/60)

    return mdl, loading_t0

In [14]:
if __name__ == '__main__':
    #data_url = 'C:/Users/20937337/Desktop/testData/data/realData/'
    #data_url = 'C:/Users/jxhua/OneDrive - Curtin University of Technology Australia/Desktop/toUpload/'
    data_url = 'D:/toUpload/'
    #bin_num = [30, 30, 30]
    bin_num = [22, 30, 35]
    grade_margin = 0.2

    model, start_t0 = build(bin_num, data_url, grade_margin)

    # Solve the model and print solution
    solving_t5 = datetime.now()
    solution, objective_value = solve(model)
    print("total running time: ", (solving_t5 - start_t0).seconds/60)
    print("The maximum profit is ", objective_value/10000)
    #print("The maximum profit is ", [i/10000 for i in obj_value])
    
    '''model.excavated = get_solution(model.excavated, solution, 'excavated', trans=True)
    model.filled = get_solution(model.filled, solution, 'filled', trans=True)
    model.t_m_b_p = get_solution(model.t_m_b_p, solution, 'tons_t_m_b_p')
    model.t_m_b_s = get_solution(model.t_m_b_s, solution, 'tons_t_m_b_s')
    model.t_m_b_d_l = get_solution(model.t_m_b_d_l, solution, 'tons_t_m_b_d_l')
    model.t_s_p = get_solution(model.t_s_p, solution, 'tons_t_s_p')
    model.t_s_d_l = get_solution(model.t_s_d_l, solution, 'tons_t_s_d_l')
    
    model.t_m_b_p.to_csv(data_url+'t_m_b_p.csv', index=False)
    model.t_m_b_s.to_csv(data_url+'t_m_b_s.csv', index=False)
    model.t_m_b_d_l.to_csv(data_url+'t_m_b_d_l.csv', index=False)
    model.t_s_p.to_csv(data_url+'t_s_p.csv', index=False)
    model.t_s_d_l.to_csv(data_url+'t_s_d_l.csv', index=False)
    model.t_m_b_d_l.loc[model.t_m_b_d_l['var_value']>0].to_csv(data_url+'t_m_b_d_l_var1.csv', index=False)'''
    print("model complete!")

data loading time:  0.5166666666666667
data preprocessing time:  0.7666666666666667
#waste stockpile non-active constraints: 0
#reserve constraints: 8168
#mining capacity constraints: 7
#processing capacity constraints: 14
#dumping capacity constraints: 101
#block precedence constraints: 30340
#dump location precedence constraints: 144
#dump location now constraints: 404
Model: mine_prod
 - number of variables: 548364
   - binary=8572, integer=0, continuous=539792
 - number of constraints: 41220
   - linear=41220
 - parameters: defaults
 - objective: none
 - problem type is: MILP
#Stockpile bin capacity constraints: 632
#Stockpile bin void flow constraints: 79
#Grade blending constraints: 8
Model: mine_prod
 - number of variables: 548364
   - binary=8572, integer=0, continuous=539792
 - number of constraints: 41939
   - linear=41939
 - parameters: defaults
 - objective: none
 - problem type is: MILP
variables setting time:  0.4
objective setting time:  0.016666666666666666
model comple

In [22]:
len(model.t_m_b_p)/4, len(model.t_m_b_s)/4, len(model.t_m_b_d_l)/4, len(model.t_s_d_l)/4, len(model.t_s_p)/4

(1448.0, 1964.0, 131182.0, 200.0, 154.0)

In [25]:
len(model.t_m_b_p)/4+len(model.t_m_b_s)/4+len(model.t_m_b_d_l)/4+len(model.t_s_d_l)+len(model.t_s_p)

136010.0

In [24]:
539792/4

134948.0

In [ ]:
#model.df_blocks.head()

len(model.df_blocks)

In [ ]:
model.df_blocks.columns

In [ ]:
dd = model.df_blocks.copy()
dd['coef'] = np.where(dd['wting']==1, 1.2, np.where(dd['wting']==2, 1.25, 1.3))
dd['volume'] = dd['quantity'] / dd['density'] * dd['coef']
dd1 = dd.loc[(dd.if_active==1) & (dd.grade<=0.26)]
paf_vol = dd1['volume'].sum()u/

In [ ]:
paf_vol/25000

In [15]:
model.excavated = get_solution(model.excavated, solution, 'excavated', trans=True)
model.filled = get_solution(model.filled, solution, 'filled', trans=True)
model.t_m_b_p = get_solution(model.t_m_b_p, solution, 'tons_t_m_b_p')
model.t_m_b_s = get_solution(model.t_m_b_s, solution, 'tons_t_m_b_s')
model.t_m_b_d_l = get_solution(model.t_m_b_d_l, solution, 'tons_t_m_b_d_l')
model.t_s_p = get_solution(model.t_s_p, solution, 'tons_t_s_p')
model.t_s_d_l = get_solution(model.t_s_d_l, solution, 'tons_t_s_d_l')

In [18]:
model.t_m_b_d_l.var_value.sum()

1563699.9999999998

In [ ]:
data_url = 'D:/toUpload/'
bin_num = [22, 30, 35]
grade_margin = 0.2

model = Model("mine_prod")
load_data(model, data_url, bin_num, grade_margin)
setup_data(model)
setup_variables(model)

In [ ]:
#len(model.t_m_b_p)+len(model.t_m_b_s)+len(model.t_m_b_d_l)+len(model.t_s_p)+len(model.t_s_d_l)+len(model.excavated)+len(model.filled)
len(model.t_m_b_p), len(model.t_m_b_s), len(model.t_m_b_d_l), len(model.t_s_p), len(model.t_s_d_l), len(model.excavated), len(model.filled)